<a href="https://colab.research.google.com/github/DeveloperSeJin/Definition-of-Pathological-Image-Analysis-for-Cancer-Cla/blob/main/UNLV%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
from google.colab import drive
drive.mount('/content/drive')
!pip install theano
import pandas as pd
import numpy as np
%cd /content/drive/MyDrive/cox-nnet
from cox_nnet import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/drive/MyDrive/cox-nnet


In [77]:
data1 = pd.read_csv('../UNLV_project/data_RNA_Seq_v2_mRNA_median_all_sample_Zscores.txt', sep='\t')
data2 = pd.read_csv('../UNLV_project/data_bcr_clinical_data_patient.txt', sep='\t')
data1 = data1.set_index('Hugo_Symbol', drop = True)

In [78]:
data1 = data1.T
data1.rename(index = lambda x : str(x).replace('-01', ''), inplace = True)
data1 = data1.iloc[1:]
data1

Hugo_Symbol,LOC100130426,UBE2Q2P3,UBE2Q2P3,HMGB1P1,TIMM23,MOXD2,LOC155060,RNU12-2P,SSX9,LOC317712,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,FLJ10821,ZZZ3,TPTEP1,AKR1C6P
TCGA-05-4244,-2.2883,0.0380,0.0691,-1.9057,-0.0395,NaN,1.0624,0.5387,-1.2501,NaN,...,-0.2092,-0.4448,-0.2293,-1.8833,0.9850,-0.4967,-0.2737,-0.4482,-1.0407,-1.2966
TCGA-05-4249,-2.2883,-0.3514,0.1971,-0.2950,0.1945,NaN,-0.0690,1.4599,-1.2501,NaN,...,0.7816,0.8157,0.4243,-0.1788,-0.1119,-0.8791,0.3183,0.0984,1.0324,-1.2966
TCGA-05-4250,-2.2883,-0.3435,-0.7239,-1.9091,0.7761,NaN,-1.4074,-2.1796,-1.2501,NaN,...,0.7041,0.0911,-0.6346,-0.0054,-1.8251,0.8248,0.3805,-0.6777,-1.0698,-1.2966
TCGA-05-4382,-2.2883,0.1873,-0.4402,-0.5333,-0.1787,NaN,0.5870,-0.6958,-0.9868,NaN,...,-0.1734,-0.0352,-0.2710,0.5825,0.3717,1.2704,0.3781,-0.0188,-0.0218,-1.2966
TCGA-05-4384,-2.2883,-1.2251,-1.3555,-0.8895,-1.1778,NaN,0.7614,-0.3706,-1.2501,NaN,...,1.3474,1.1341,0.7826,-0.7635,-0.1493,-0.1648,1.2559,0.2385,1.5166,-1.2966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-NJ-A55O,-2.2883,0.5729,1.0176,-0.0218,0.0408,NaN,0.6719,0.7543,-1.2501,NaN,...,-0.1517,-0.1613,0.2250,-0.2732,-0.3860,0.2930,0.6698,-0.3969,1.7447,-1.2966
TCGA-NJ-A55R,-2.2883,-0.1679,-0.0462,-0.8099,-0.3206,NaN,1.1540,-0.1200,-1.2501,NaN,...,0.3282,0.1494,1.1546,0.4441,-0.2421,-0.3319,1.5561,-0.2637,1.9706,0.5663
TCGA-NJ-A7XG,-2.2883,1.8645,2.7613,-0.4522,-0.6611,NaN,1.0220,-2.1796,-1.2501,NaN,...,-2.0722,-1.7646,0.5444,-0.6192,-1.4443,-1.2127,0.8153,-1.3431,-1.1463,-1.2966
TCGA-O1-A52J,-2.2883,0.4532,1.0870,-1.3473,0.7679,NaN,0.1428,2.9555,-1.2501,NaN,...,2.5448,1.1317,0.7244,-1.9766,-0.1344,0.4326,0.9605,-1.0035,1.0542,-0.3735


In [79]:
data2 = data2.iloc[4:]
data2 = data2[['Patient Identifier', 'Overall Survival Status', 'Overall Survival (Months)']]
data2 = data2.set_index('Patient Identifier', drop = True)
data2

,Overall Survival Status,Overall Survival (Months)
Patient Identifier,,
TCGA-05-4244,0:LIVING,0
TCGA-05-4245,0:LIVING,23.98
TCGA-05-4249,0:LIVING,50.03
TCGA-05-4250,1:DECEASED,3.98
TCGA-05-4382,0:LIVING,19.94
...,...,...
TCGA-NJ-A55O,0:LIVING,0.43
TCGA-NJ-A55R,0:LIVING,19.81
TCGA-NJ-A7XG,0:LIVING,20.27


In [80]:
data = data2.merge(data1, left_on = 'Patient Identifier', right_index = True)

In [81]:
missing_data = data.isnull().sum() / len(data) > 0.8

missing_data = missing_data.to_frame()
missing_data.loc[missing_data[0] == True]

,0
MOXD2,True
LOC317712,True
SPATA31B1P,True
REXO1L6P,True
PPBPL1,True
...,...
UNCX,True
VTRNA1-1,True
VTRNA1-2,True
VTRNA1-3,True


In [82]:
missing_data_index = missing_data.loc[missing_data[0] == True, :].index
data = data.drop(missing_data_index, axis = 1)
data

,Overall Survival Status,Overall Survival (Months),LOC100130426,UBE2Q2P3,UBE2Q2P3,HMGB1P1,TIMM23,LOC155060,RNU12-2P,SSX9,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,FLJ10821,ZZZ3,TPTEP1,AKR1C6P
Patient Identifier,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244,0:LIVING,0,-2.2883,0.0380,0.0691,-1.9057,-0.0395,1.0624,0.5387,-1.2501,...,-0.2092,-0.4448,-0.2293,-1.8833,0.9850,-0.4967,-0.2737,-0.4482,-1.0407,-1.2966
TCGA-05-4249,0:LIVING,50.03,-2.2883,-0.3514,0.1971,-0.2950,0.1945,-0.0690,1.4599,-1.2501,...,0.7816,0.8157,0.4243,-0.1788,-0.1119,-0.8791,0.3183,0.0984,1.0324,-1.2966
TCGA-05-4250,1:DECEASED,3.98,-2.2883,-0.3435,-0.7239,-1.9091,0.7761,-1.4074,-2.1796,-1.2501,...,0.7041,0.0911,-0.6346,-0.0054,-1.8251,0.8248,0.3805,-0.6777,-1.0698,-1.2966
TCGA-05-4382,0:LIVING,19.94,-2.2883,0.1873,-0.4402,-0.5333,-0.1787,0.5870,-0.6958,-0.9868,...,-0.1734,-0.0352,-0.2710,0.5825,0.3717,1.2704,0.3781,-0.0188,-0.0218,-1.2966
TCGA-05-4384,0:LIVING,13.99,-2.2883,-1.2251,-1.3555,-0.8895,-1.1778,0.7614,-0.3706,-1.2501,...,1.3474,1.1341,0.7826,-0.7635,-0.1493,-0.1648,1.2559,0.2385,1.5166,-1.2966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-NJ-A55O,0:LIVING,0.43,-2.2883,0.5729,1.0176,-0.0218,0.0408,0.6719,0.7543,-1.2501,...,-0.1517,-0.1613,0.2250,-0.2732,-0.3860,0.2930,0.6698,-0.3969,1.7447,-1.2966
TCGA-NJ-A55R,0:LIVING,19.81,-2.2883,-0.1679,-0.0462,-0.8099,-0.3206,1.1540,-0.1200,-1.2501,...,0.3282,0.1494,1.1546,0.4441,-0.2421,-0.3319,1.5561,-0.2637,1.9706,0.5663
TCGA-NJ-A7XG,0:LIVING,20.27,-2.2883,1.8645,2.7613,-0.4522,-0.6611,1.0220,-2.1796,-1.2501,...,-2.0722,-1.7646,0.5444,-0.6192,-1.4443,-1.2127,0.8153,-1.3431,-1.1463,-1.2966


In [83]:
dup_cols = data.columns[data.columns.duplicated()]
dup_indices = [ [i, col] for i, col in enumerate(data.columns) if col in dup_cols]
print(dup_indices)
dup_indices.sort(key=lambda x:x[1])
dup_indices = list(list(zip(*dup_indices))[0])
indexes = []
for i in range(0, len(dup_indices), 2):
    pair = data.iloc[:, dup_indices[i:i+2]]
    variances = pair.var()   
    if variances.iloc[0] < variances.iloc[1]:
        indexes.append(dup_indices[i])        
    else:
        indexes.append(dup_indices[i+1])
print(indexes)
data = data.iloc[:, [i for i in range(data.shape[1]) if i not in indexes]]
print(data)

[[3, 'UBE2Q2P3'], [4, 'UBE2Q2P3'], [492, 'PALM2AKAP2'], [936, 'SH3D20'], [1679, 'NEBL'], [1691, 'CC2D2B'], [1715, 'CCDC7'], [1828, 'PLEKHG7'], [2252, 'C1orf84'], [2545, 'C5orf23'], [3050, 'CC2D2B'], [3179, 'CCDC7'], [4605, 'CYorf15B'], [4606, 'CYorf15B'], [6449, 'QSOX1'], [6473, 'LINC00875'], [8898, 'C1orf84'], [9713, 'LINC00875'], [10017, 'NKAIN3'], [11516, 'NBPF16'], [11517, 'NBPF16'], [11611, 'NCRNA00185'], [11686, 'NEBL'], [11826, 'NKAIN3'], [11989, 'C5orf23'], [12756, 'PALM2AKAP2'], [13452, 'PLEKHG7'], [14321, 'QSOX1'], [15798, 'SH3D20'], [18502, 'NCRNA00185']]
[2252, 11989, 3050, 3179, 4606, 9713, 11516, 18502, 11686, 11826, 12756, 13452, 6449, 15798, 4]
                   Overall Survival Status Overall Survival (Months)  \
Patient Identifier                                                     
TCGA-05-4244                      0:LIVING                         0   
TCGA-05-4249                      0:LIVING                     50.03   
TCGA-05-4250                    1:DECEASED 

In [85]:
data['Overall Survival Status'] = data['Overall Survival Status'].replace({'1:DECEASED': 1, '0:LIVING': 0})
data = data.rename(columns={'Overall Survival (Months)': 't', 'Overall Survival Status': 'e'})
data['t'] = data['t'].replace({'[Not Available]': 0})
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

<ipython-input-85-274e46b474bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Overall Survival Status'] = data['Overall Survival Status'].replace({'1:DECEASED': 1, '0:LIVING': 0})


In [92]:
train = train.reset_index()
test = test.reset_index()
t_train = train['t'].to_numpy(dtype='float')
e_train = train['e']
t_test = test['t'].to_numpy(dtype='float')
e_test = test['e']
train = train.drop(['e','t','Patient Identifier'],axis=1)
test = test.drop(['e','t','Patient Identifier'],axis=1)
X_train = train.to_numpy(dtype=float)
X_test = train.to_numpy(dtype=float)

In [ ]:
#model_params = dict(node_map = None, input_split = None)
#search_params = dict(method = "nesterov", learning_rate=0.01, momentum=0.9,
#   max_iter=2000, stop_threshold=0.995, patience=1000, patience_incr=2, rand_seed = 123,
#    eval_step=23, lr_decay = 0.9, lr_growth = 1.0)
#cv_params = dict(cv_seed=1, n_folds=5, cv_metric = "loglikelihood",
#    L2_range = numpy.arange(-4.5,1,0.5))

In [95]:
X_train = train.to_numpy(dtype=float)
model, loss = trainCoxMlp(x_train = X_train, ytime_train = t_train ,ystatus_train = e_train)

Using nesterov accelerated gradient
training model
running time: 775.715060 seconds
total iterations: 2898.000000


In [104]:
y_test = model.predictNewData(X_test)

In [116]:
test

,index,LOC100130426,UBE2Q2P3,HMGB1P1,TIMM23,LOC155060,RNU12-2P,SSX9,EZHIP,EFCAB8,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,FLJ10821,ZZZ3,TPTEP1,AKR1C6P
0,0,-2.2883,-3.0165,-0.9737,0.1582,-1.8041,-2.1796,-1.2501,-0.9569,-1.6633,...,0.3504,0.7936,-0.9703,-1.7240,0.6790,1.3564,-1.1038,-0.8139,-0.7172,-1.2966
1,1,-2.2883,1.7115,-0.5379,-1.2639,0.8677,-2.1796,-1.2501,-0.5799,-0.5735,...,0.2891,-0.0797,1.4630,-0.9288,-1.3965,-0.1103,1.5929,-1.1709,0.9504,-0.4011
2,2,-2.2883,-0.1229,-0.0651,-0.5350,-0.1231,0.2414,-1.2501,-0.5900,0.0450,...,0.3173,0.0750,0.0731,0.0728,1.2190,0.8884,1.2299,0.6188,0.5017,-1.2966
3,3,-2.2883,-0.7247,1.5892,-0.5943,0.3360,-0.4937,-1.2501,-0.2105,0.5615,...,-0.4082,-0.1701,-1.1218,-1.4482,-0.1035,2.3496,1.1654,-1.1301,0.5908,-1.2966
4,4,-2.2883,-0.9512,0.5325,-1.3480,0.6618,-2.1796,-1.2501,-0.3330,1.3895,...,0.5800,-0.1281,0.2525,-0.9176,-0.2294,0.0536,1.1436,-0.3176,0.2224,-1.2966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,98,-2.2883,-0.3501,-0.9236,-0.2210,1.2964,0.7822,-1.2501,-0.9569,0.0381,...,-0.0901,-0.9980,0.0952,0.7776,0.3707,-0.7352,-0.6518,0.3754,-0.7315,-1.2966
99,99,-2.2883,1.0630,-0.2081,0.2432,0.4341,-2.1796,-1.2501,-0.1985,0.3221,...,-0.3578,0.3235,0.9544,1.2821,0.3791,-1.3258,0.2967,1.4097,-0.7204,-1.2966
100,100,-2.2883,0.3975,-0.6108,-0.8910,0.7647,0.0889,-1.2501,-0.9569,0.4039,...,0.8700,-0.6518,1.2078,0.2232,0.4778,-0.6557,0.1516,-0.2103,1.1850,-1.2966
101,101,-2.2883,-1.5393,0.0196,0.1793,1.6509,-2.1796,-1.2501,-0.2599,0.1864,...,-0.7068,-0.0050,1.1994,-1.6875,-0.2386,0.2105,-0.1859,0.0259,-0.6336,-1.2966


In [110]:
import pickle

def saveModels(model, file_name):
    b = [tvar.eval() for tvar in model.b]
    W = [tvar.eval() for tvar in model.W]
    node_map = model.node_map
    input_split = model.input_split
    n_samples = model.n_samples
    rng = model.rng
    x_train = model.x_train

    with open(file_name, "wb") as f:
        pickle.dump((W, b, node_map, input_split, n_samples, x_train, rng), f)

saveModels(model,"model.txt")